In [1]:
import numpy as np
np.set_printoptions(suppress=True, precision=6)

In [2]:
from robot_model import KR6R900sixx, UR5

In [3]:
ur5 = UR5()

Renderer(camera=PerspectiveCamera(aspect=1.5, children=(DirectionalLight(color='white', intensity=0.66, positi…

In [4]:
# ur5.q = np.array([0,np.pi/2,np.pi/2,np.pi/2,-np.pi/2,0])

In [5]:
ur5.q = np.array([0,0,0,0,0,0])

In [23]:
ur5._ee_debug_call_twice()

array([[-1.      , -0.      ,  0.      ,  0.81725 ],
       [ 0.      ,  0.      ,  1.      ,  0.19145 ],
       [-0.      ,  1.      , -0.      , -0.005491],
       [ 0.      ,  0.      ,  0.      ,  1.      ]])

In [34]:
ur_params = [DenavitHartenbergFrame(0.0,     np.pi/2,  0.089159,      0.0),
             DenavitHartenbergFrame(-0.425,      0.0,     0.0,   -np.pi/2),
             DenavitHartenbergFrame(-0.39225,    0.0,     0.0,        0.0),
             DenavitHartenbergFrame(0.0  ,   np.pi/2,   0.10915, -np.pi/2),
             DenavitHartenbergFrame(0.0  ,  -np.pi/2,   0.09465,      0.0),
             DenavitHartenbergFrame(0.0  ,       0.0,   0.0823,       0.0)]
ur_joint_direction = np.array([-1, -1, 1, 1, 1, 1])

In [35]:
ur_fk_solver = DenavitHartenbergFKSolver(ur_params)
T, _, J = ur_fk_solver.solve(np.array([0,0,0,0,0,0]))
print(T)

[[-1.        0.        0.       -0.      ]
 [-0.       -0.       -1.       -0.19145 ]
 [-0.       -1.        0.        1.001059]
 [ 0.        0.        0.        1.      ]]


In [11]:
agilus = KR6R900sixx(interact=False)

Renderer(camera=PerspectiveCamera(aspect=1.5, children=(DirectionalLight(color='white', intensity=0.66, positi…

In [12]:
agilus.q = [0.4,-np.pi/2 + 1, np.pi/2, 1, 1, 0]

In [11]:
class DenavitHartenbergFrame:
    def __init__(self, a, alpha, d, theta):
        self.a = a
        self.alpha = alpha
        self.d = d
        self.theta = theta
        
    def __repr__(self):
        return 'DH(a={}, alpha={}, d={}, theta={})'.format(self.a, self.alpha, self.d, self.theta)

In [12]:
dh_params = [DenavitHartenbergFrame(0.025, -np.pi/2,  0.4, 0),
             DenavitHartenbergFrame(0.455,      0.0,  0.0, 0),
             DenavitHartenbergFrame(0.035, -np.pi/2,  0.0, -np.pi/2),
             DenavitHartenbergFrame(0.0  ,  np.pi/2, 0.42, 0),
             DenavitHartenbergFrame(0.0  , -np.pi/2,  0.0, 0),
             DenavitHartenbergFrame(0.0  ,      0.0, 0.08, np.pi)]

In [13]:
class DenavitHartenbergFKSolver:
    def __init__(self, dh_params, rotation_directions=None):
        self._dh_params = dh_params
        self._rotation_directions = rotation_directions
        
    def _dh_transform(self, a, alpha, d, theta):
        ct = np.cos(theta)
        st = np.sin(theta)
        ca = np.cos(alpha)
        sa = np.sin(alpha)
        return np.array([[ct, -st * ca,  st * sa, a * ct],
                         [st,  ct * ca, -ct * sa, a * st],
                         [0.0,      sa,       ca, d],
                         [0.0,     0.0,      0.0, 1.0]])
    
    @property
    def dh_params(self):
        return self._dh_params
    
    def solve(self, q):
        if self._rotation_directions is not None:
            q *= self._rotation_directions
        intermediate_link_transforms = []
        for dhi, qi in zip(self._dh_params, q):
            intermediate_link_transforms.append(
                self._dh_transform(dhi.a, dhi.alpha, dhi.d, (dhi.theta + qi)))
        link_transforms = [intermediate_link_transforms[0]] # T^0_1
        trf_0_i = intermediate_link_transforms[0]
        for i in range(1, 6):
            trf_0_i = trf_0_i.dot(intermediate_link_transforms[i])
            link_transforms.append(trf_0_i)
            

        p0 = np.array([0, 0, 0])
        z0 = np.array([0, 0, 1])
        pn = link_transforms[-1][:3, 3]
        J = np.zeros((6, 6))
        J[:3, 0] = np.cross(z0, pn-p0)
        J[3:, 0] = z0
        for i in range(5):
            T = link_transforms[i]
            zi = T[:3, 2]
            pi = T[:3, 3]
            J[:3, i+1] = np.cross(zi, pn-pi)
            J[3:, i+1] = zi    
        
        return (link_transforms[-1], link_transforms, J)
        

In [12]:
fk_solver = DenavitHartenbergFKSolver(dh_params)
trf, _, jac = fk_solver.solve(agilus.q)
print(trf)

NameError: name 'agilus' is not defined

In [10]:
ee_debug_trf = agilus._ee_debug_call_twice()
ee_debug_trf

array([[-0.8221,  0.4418, -0.3592,  0.5831],
       [ 0.1461,  0.7734,  0.6169,  0.308 ],
       [ 0.5503,  0.4546, -0.7003,  0.2553],
       [ 0.    ,  0.    ,  0.    ,  1.    ]])

In [17]:
np.allclose(trf, ee_debug_trf)

True

In [18]:
class KR6R900sixxIKSolver:
    def __init__(self, fk_solver):
        self._fk_solver = fk_solver
        
    def solve_analytical(self, trf, front=1, elbow=1, wrist=1):
        ne = trf[:3,0]
        se = trf[:3,1]
        ae = trf[:3,2]
        pe = trf[:3,3]
        
        d1 = self._fk_solver.dh_params[0].d
        d4 = self._fk_solver.dh_params[3].d
        d6 = self._fk_solver.dh_params[5].d
        
        a1 = self._fk_solver.dh_params[0].a
        a2 = self._fk_solver.dh_params[1].a
        a3 = self._fk_solver.dh_params[2].a
        
        l3 = np.sqrt(a3 * a3 + d4 * d4)
        
        pw = pe - d6 * ae
        
        q1 = np.arctan2(front * pw[1], front*pw[0])
        
        pty = - (pw[2] - d1)
        ptx = front * (np.sqrt(pw[0] * pw[0] + pw[1] * pw[1]) - front * a1)
        
        c3 = (ptx * ptx + pty * pty - a2 * a2 - l3 * l3 ) / (2 * a2 * l3)
        s3 = elbow * np.sqrt(1.0 - c3 * c3)
        psi3 = np.arctan2(s3, c3)
        offset3 = np.arctan2(a3, d4)
        q3 = psi3 + offset3
        
        den2 = (ptx * ptx + pty * pty)
        c2 = (ptx * (a2 + l3 * c3) + pty * l3 * s3) / den2
        s2 = (pty * (a2 + l3 * c3) - ptx * l3 * s3) / den2
        q2 = np.arctan2(s2, c2)
        
        _, trfs, _ = self._fk_solver.solve(np.array([q1, q2, q3, 0.0, 0.0, 0.0]))
        
        rot_0_3 = trfs[2][:3,:3]
        rot_3_6 = rot_0_3.T.dot(trf[:3,:3])
        
        q4 = np.arctan2(wrist * rot_3_6[1,2], wrist * rot_3_6[0,2])
        q5 = -np.arctan2(wrist * np.sqrt(rot_3_6[0,2] * rot_3_6[0,2] + rot_3_6[1,2] * rot_3_6[1,2]), rot_3_6[2,2])
        q6 = np.arctan2(wrist * rot_3_6[2,1], -wrist * rot_3_6[2,0]) + self._fk_solver.dh_params[-1].theta
        
        return np.array([q1, q2, q3, q4, q5, q6])
    

    def solve_iterative(self, trf, q0):
        Rd = trf[:3,:3]
        qk = q0

        for i in range(1000):
            Tk, _, Jk = fk_solver.solve(qk)
            Rk = Tk[:3,:3]
            Re = Rd @ Rk.T
            ep = trf[:3,3] - Tk[:3,3]
            eo = np.array([Re[2,1] - Re[1,2], 
                           Re[0,2]-  Re[2,0], 
                           Re[1,0] - Re[0,1]]) * 0.5
            
            e = np.array((ep, eo)).reshape(6,1)

            K = 0.1
            dq = K * np.dot(np.linalg.pinv(Jk),e).ravel()
            qk = qk + dq 

            if np.linalg.norm(e) < 1e-6:
                print(i)
                break
        return qk

In [20]:
fk_solver = DenavitHartenbergFKSolver(dh_params)
q1 = agilus.q
print(q1)
trf1, _, _ = fk_solver.solve(q1)
ik_solver = KR6R900sixxIKSolver(fk_solver)
q2 = ik_solver.solve_analytical(trf, front=1, wrist=-1)
print(q2)
q3 = ik_solver.solve_iterative(trf, np.array([1,1,1,1,1,1]))
print(q3)

trf2, _, _ = fk_solver.solve(q2)
trf3, _, _ = fk_solver.solve(q3)

[ 0.4    -0.5708  1.5708  1.      1.      0.    ]
[ 0.4    -0.5708  1.5708  1.      1.      0.    ]
140
[ 0.4    -0.5708  1.5708  1.      1.      0.    ]


In [21]:
agilus.q

array([ 0.4   , -0.5708,  1.5708,  1.    ,  1.    ,  0.    ])

In [22]:
agilus._ee_debug_call_twice()

array([[-0.8221,  0.4418, -0.3592,  0.5831],
       [ 0.1461,  0.7734,  0.6169,  0.308 ],
       [ 0.5503,  0.4546, -0.7003,  0.2553],
       [ 0.    ,  0.    ,  0.    ,  1.    ]])

In [23]:
trf1

array([[-0.8221,  0.4418, -0.3592,  0.5831],
       [ 0.1461,  0.7734,  0.6169,  0.308 ],
       [ 0.5503,  0.4546, -0.7003,  0.2553],
       [ 0.    ,  0.    ,  0.    ,  1.    ]])